#  Evaluacion de los modelos

#### 1. Import all libraries

In [1]:
from __future__ import unicode_literals, print_function 
import xml.etree.ElementTree as ET     
import pandas as pd
import spacy
from spacy.gold import GoldParse
from spacy.scorer import Scorer
from spacy.tokens import Span

#### 2. Load the whole Gold Standards

In [3]:
test_data=[]   

for tn in range (1,153):  
    
    anot_path = '/Users/.../dataset_k10/fold_10/annot/a_fold10_'+str(tn)+'.xml' 
    tree = ET.parse(anot_path)                    
    root = tree.getroot()
    
    df_cols = ["Start", "End", "Type"]                                 
    rows = []
    i=0
    for node in root.iter('Annotation'):
        s_Id = node.attrib.get("Id")
        s_type = node.attrib.get("Type")
        s_start = int(node.attrib.get("StartNode"))
        s_end = int(node.attrib.get("EndNode"))
        i+=1      
        rows.append({"Start": s_start, "End": s_end, "Type": s_type})  
        
    out_df = pd.DataFrame(rows, columns = df_cols)
    out_df = out_df[out_df.Type != 'paragraph']                       

    records = out_df.to_records(index=False)
    annot = list(records)                                            

    text_path = '/Users/.../dataset_k10/fold_10/txt/t_fold10_'+str(tn)+".txt"
    texto =open(text_path, "r")                                      
    texto =texto.read() 
    texto = texto.replace('\n','')                                 
    texto = texto.replace('\t','')                              
    TRAIN_DATA_min = (texto, {'entities':annot})
    test_data.append(TRAIN_DATA_min)

#### 3. Evaluate Function

In [4]:
def evaluate(model, examples):
    scorer = Scorer()
    for input_, annot in examples:
        doc_gold_text = model.make_doc(input_)
        gold = GoldParse(doc_gold_text, entities=annot['entities'])
        pred_value = model(input_)
        scorer.score(pred_value, gold)
    return scorer.scores

#### 4. Scores

In [5]:
nlp = spacy.load('/Users/.../dataset_k10/modelos/modelo_k10_sm_dr_1/it_8')
results_all = evaluate(nlp, test_data)
print(results_all)   

{'uas': 0.0, 'las': 0.0, 'ents_p': 68.33013435700576, 'ents_r': 39.77653631284917, 'ents_f': 50.28248587570621, 'ents_per_type': {'SCITERM': {'p': 66.66666666666666, 'r': 25.64102564102564, 'f': 37.03703703703704}, 'MEDTERM': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'DISEASE': {'p': 80.32786885245902, 'r': 80.32786885245902, 'f': 80.32786885245902}, 'PER': {'p': 76.47058823529412, 'r': 61.904761904761905, 'f': 68.42105263157895}, 'JOB': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'TIME': {'p': 45.45454545454545, 'r': 25.0, 'f': 32.25806451612903}, 'ORG': {'p': 72.14285714285714, 'r': 72.66187050359713, 'f': 72.40143369175627}, 'DATE': {'p': 60.0, 'r': 67.74193548387096, 'f': 63.63636363636363}, 'EVENT': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'PRODUCT': {'p': 0.0, 'r': 0.0, 'f': 0.0}, 'GPE': {'p': 84.7457627118644, 'r': 64.93506493506493, 'f': 73.52941176470588}, 'QUANT': {'p': 63.26530612244898, 'r': 51.66666666666667, 'f': 56.880733944954144}, 'ADDRESS': {'p': 20.0, 'r': 33.33333333333333, 'f': 25.0}, 'LOC': {'p